# Grocery Problem: Preparing Data
In this notebook, we preprocess the dataset for machine learning

## Imports & setup envrionment 

In [1]:
%load_ext autoreload
%autoreload 2 

%matplotlib inline

In [2]:
import gc
import time
import pickle
import numpy as np
import matplotlib as plt
import pandas as pd

from datetime import date

from fastai.imports import *
from fastai.structured import *

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display # extract a feature record from each date
from sklearn import metrics
from sklearn import preprocessing

from multiprocessing import cpu_count, Pool
from multiprocessing.pool import ThreadPool

In [3]:
DEBUG_MODE = True

## Utilities
Some utilities to make it easier/faster to work with pandas 
##### parallelise pandas operations to better load the cpu


In [4]:
# Parallel apply the given function to the given dataframe.
# Splits the dataframe into  the given number of splits 
# and applys the function to each split concurrently on n_procs processes
def parallel_apply(func, df, n_splits=cpu_count() * 8, n_procs=cpu_count()):
    # Split dataframe into n_split splits
    df_splits = np.array_split(df, n_splits)
    # Apply the function using process pool
    pool = Pool(processes=n_procs)
    df_splits = pool.map(func, df_splits)
    pool.close()
    # Merge results into single dataframe
    df = pd.concat(df_splits)
    
    return df

## Preprocess data

### Load Data
Load the data processed from the previous (grocery_problem_data_preprocessing.ipynb)

In [5]:
PATH = "tmp/groceries/"
df_train = pd.read_feather(os.path.join(PATH, "train.feather"))
df_test = pd.read_feather(os.path.join(PATH, "test.feather"))

print(f"Loaded {len(df_train)} training examples, {len(df_test)} test examples")

Loaded 125497040 training examples, 3370464 test examples


In [6]:
df_train.sample(100000).describe(include="all").T

,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
id,100000,NaN,NaN,NaN,NaN,NaN,6.27324e+07,3.61029e+07,922,3.15609e+07,6.25638e+07,9.40351e+07,1.25495e+08
date,100000,1682,2016-12-23 00:00:00,109,2013-01-02 00:00:00,2017-08-15 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
store_nbr,100000,NaN,NaN,NaN,NaN,NaN,27.5094,16.3756,1,12,28,43,54
item_nbr,100000,NaN,NaN,NaN,NaN,NaN,973702,519555,96995,551893,959500,1.35397e+06,2.12219e+06
unit_sales,100000,NaN,NaN,NaN,NaN,NaN,8.72572,29.3038,-1,2,4,9,7001
onpromotion,100000,1,True,100000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
family,100000,32,GROCERY I,36656,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
class,100000,NaN,NaN,NaN,NaN,NaN,1955.76,1140.29,1002,1048,1190,2712,7780
perishable,100000,NaN,NaN,NaN,NaN,NaN,0.25229,0.434329,0,0,0,1,1
city,100000,22,Quito,41746,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Drop Nans in the dataset
We have introduced Nans in the dataset when merging the different dataframes.

To solve this we drop the Nan's we have added in the dataset:

In [7]:
def rm_nan(df): return df.dropna()

%time df_train = parallel_apply(rm_nan, df_train)
%time df_test = rm_nan(df_test)
gc.collect()

CPU times: user 13.6 s, sys: 30 s, total: 43.7 s
Wall time: 41.8 s
CPU times: user 789 ms, sys: 393 ms, total: 1.18 s
Wall time: 1.92 s


187

### Feature Selection
More operations on such a large dataset are going to be incredibly expensive. Things we can do:
1. Parallelise and scale to process all the data (dask, spark, hadoop)
2. Sample dataset to get most important data, resulting in a computational  managable sample 

Picking option 2, we sample the datset.
one option is to randomly sample the dataset.
However, some heursitics can give us a better sample.
The task of the competition is to predict the next two weeks after the end of the training set. Intutively, we sort by date and sample the latests data:


In [8]:
# Sample latests data
SAMPLE_SIZE = 10 * int(1e+6) # sample 10 million
# due to a previous operation, we have already sorted the data
#df_train.sort_values(by="date", inplace=True) 
df_train = df_train.tail(SAMPLE_SIZE)

### Feature Extraction
Taking a look at the dataset, there are still some feature extraction we need before we can fit a model

1. Extract numeric features from the the `date` column
2. Convert categorical features into numeric features
3. Convert boolean features into 0 and 1s.

#### Feature extraction of the date column
One way to apporach this is convert date to the number of days from the epoch (ie the first date in the dataset).

In [9]:
# Extract date day offsets feature for the date series
# Returns the extracted day offests series
def extract_date_feature(dates):
    min_date = dates.min()
    day_offsets = dates.apply((lambda d:(d - min_date).days))
    return day_offsets

# Add a date day offsets feature to the  given dataframe
# derieved from the dates on the given column name col_name
# Returns the dataframe with the date day offsets feature as date
def convert_date_feature(df, col_name):
    dates =  df[col_name]
    day_offsets = parallel_apply(extract_date_feature, dates)
    #day_offsets = extract_date_feature(dates)
    new_df = df.copy()
    new_df[col_name] = day_offsets
    return new_df

In [10]:
if DEBUG_MODE:
    df = df_train.tail(int(1e+6))
    %timeit convert_date_feature(df, "date")

5.02 s ± 181 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%time df_train = convert_date_feature(df_train, "date")
%time df_test = convert_date_feature(df_test, "date")
gc.collect()

CPU times: user 436 ms, sys: 2.68 s, total: 3.11 s
Wall time: 30.7 s
CPU times: user 219 ms, sys: 2.29 s, total: 2.51 s
Wall time: 12.4 s


21

#### Convert categorical features to numerical
Convert categorical classes to numerical features

In this cause, we will be using the one hot encoding method to encoded the  features

In [12]:
# Convert the categorical columns in the dataframe in place 
# to one encoded categories
def convert_one_hot(df):
    # Find names of categorical columns
    cat_columns = [ column for column in df.columns
                   if df[column].dtype.__class__ is pd.CategoricalDtype]

    # Encode categorical variables into categorical variables
    encoded_df = df.copy()
    for column in cat_columns:
        encoding = pd.get_dummies(df[column], prefix=column)
        # remove original column and replace with one hot encoding
        encoded_df = encoded_df.drop(columns=[column])
        encoded_df = pd.concat([encoded_df, encoding], axis=1)

    
    return encoded_df

In [13]:
if DEBUG_MODE:
    df = df_train.sample(int(1e+6))
    %timeit convert_one_hot(df)

1.04 s ± 31.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
%time df_train = convert_one_hot(df_train)
%time df_test = convert_one_hot(df_test)
gc.collect()

CPU times: user 5.99 s, sys: 6.05 s, total: 12 s
Wall time: 12 s
CPU times: user 2.15 s, sys: 1.61 s, total: 3.76 s
Wall time: 3.76 s


202

#### Convert Boolean features to numeric
Finally, we convert the Boolean features from (True, False) to (1, 0) respectively

In [15]:
# converts the Boolean features from (True, False) to (1, 0) respectively
def convert_boolean(df):
    # Find boolean columns
    bool_columns = [ column for column in df.columns
                    if df[column].dtype.name == "bool" ]

    # Convert boolean columns to numeric integers
    for column in bool_columns:
        df[column] = df[column].astype(int)
        
    return df

In [16]:
if DEBUG_MODE:
    df = df_train.sample(int(1e+6))
    %timeit convert_boolean(df)

806 µs ± 7.91 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [17]:
%time df_train = convert_boolean(df_train)
%time df_test = convert_boolean(df_test)
gc.collect()

CPU times: user 74.1 ms, sys: 0 ns, total: 74.1 ms
Wall time: 71.3 ms
CPU times: user 23 ms, sys: 0 ns, total: 23 ms
Wall time: 22.8 ms


49

## Feature Scaling
Some Machine Learning algorihms perform poorly on unscaled data.So we scale the features in the data by 1. Zeroing the mean and 2. Subtracting the standard deviation

We will do this with sckit-learn's `StandardScaler`

In [18]:
# Create and fit to training data
scaler = StandardScaler(with_mean=True, with_std=True, copy=False)
%time scaler.fit(df_train.drop(columns="unit_sales"))

/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype uint8, int8, int16, float32, int32, int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)


CPU times: user 11.9 s, sys: 1min 12s, total: 1min 24s
Wall time: 1min 49s


StandardScaler(copy=False, with_mean=True, with_std=True)

In [19]:
# Scale both the training and test data
def scale_features(df):
    unit_sales = None
    if "unit_sales" in df.columns:
        unit_sales = df["unit_sales"]
        df = df.drop(columns="unit_sales")
     
    scaled_feats = scaler.transform(df)
    df = pd.DataFrame(scaled_feats, index=df.index, columns=df.columns)
    
    if not unit_sales is None:
        df = pd.concat([df, unit_sales], axis=1)
    return df

In [20]:
%time df_train = scale_features(df_train)
%time df_test = scale_features(df_test)
gc.collect()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: DataConversionWarning: Data with input dtype uint8, int8, int16, float32, int32, int64 were all converted to float64 by StandardScaler.
  


CPU times: user 4.66 s, sys: 15 s, total: 19.6 s
Wall time: 26.5 s


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: DataConversionWarning: Data with input dtype uint8, int8, int16, float32, int32, int64 were all converted to float64 by StandardScaler.
  


CPU times: user 1.1 s, sys: 2.38 s, total: 3.48 s
Wall time: 6.71 s


74

## Finishing Up and Saving data
We are finished with preprocessing the data.

Commit the data to disk for training models

In [21]:
# Reset index to because if not feather will complain
df_train = df_train.reset_index()
df_test = df_test.reset_index()    

# commit to disk in feather format
df_train.to_feather(os.path.join(PATH, "train_pp.feather"))
df_test.to_feather(os.path.join(PATH, "test_pp.feather"))

# save fitted scaler for later use
with open(os.path.join(PATH, "scaler.pickle"), "wb") as f:
    pickle.dump(scaler, f)